# Experiments Local Test



In [ ]:
import json
from collections import defaultdict

In [ ]:
def testJson(model, iterations):
    cnts = defaultdict(int)
    %env model=$model
    for i in range(iterations):
        res = !curl -s http://0.0.0.0:9000/v2/models/${model}/infer -H "Content-Type: application/json" -H "seldon-model: ${model}"\
            -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'
        res = json.loads(res[0])
        cnts[res["model_name"]] += 1
    for name,cnt in cnts.items():
        print(name,cnt)

def testGrpc(model, iterations):
    cnts = defaultdict(int)
    %env model=$model
    for i in range(iterations):
        res = !grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
              -plaintext \
              -import-path ../../apis \
              -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
              -rpc-header "seldon-model: ${model}" \
              0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer
        res = json.loads(''.join(res))
        cnts[res["modelName"]] += 1       
    for name,cnt in cnts.items():
        print(name,cnt)
        
def testExperimentJson(experiment, iterations):
    cnts = defaultdict(int)
    %env model=$experiment
    for i in range(iterations):
        res = !curl -s http://0.0.0.0:9000/v2/models/exp1/infer -H "Content-Type: application/json" -H "seldon-model: ${model}.experiment"\
            -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'
        try:
            res = json.loads(res[0])
            cnts[res["model_name"]] += 1
        except:
            print("No results returned")
            break
    for name,cnt in cnts.items():
        print(name,cnt)
        
def testExperimentGrpc(experiment, iterations):
    cnts = defaultdict(int)
    %env model=$experiment
    for i in range(iterations):
        res = !grpcurl -d '{"model_name":"exp1","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
              -plaintext \
              -import-path ../../apis \
              -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
              -rpc-header "seldon-model:${model}.experiment" \
              0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer
        res = json.loads(''.join(res))
        cnts[res["modelName"]] += 1       
    for name,cnt in cnts.items():
        print(name,cnt)

## Experiment over two iris model with default model iris

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris2"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!grpcurl -d '{"experiment":{"name":"exp1", \
               "defaultModel":"iris", \
                "candidates":[\
                    {"modelName":"iris","weight":50},\
                    {"modelName":"iris2","weight":50}\
                    ]}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StartExperiment

In [ ]:
testJson("iris",50)

In [ ]:
testGrpc("iris",50)

In [ ]:
testExperimentJson("exp1", 50)

In [ ]:
!grpcurl -d '{"name":"exp1"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StopExperiment

In [ ]:
testJson("iris", 50)

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris2"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

## Experiment over two iris models

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris2"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!grpcurl -d '{"experiment":{"name":"exp1", \
                "candidates":[{"modelName":"iris","weight":50},{"modelName":"iris2","weight":50}]}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StartExperiment

In [ ]:
testExperimentJson("exp1", 50)

In [ ]:
testExperimentGrpc("exp1", 50)

In [ ]:
!grpcurl -d '{"name":"exp1"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StopExperiment

In [ ]:
testExperimentJson("exp1", 50)

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris2"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

## Test Experiment created before models

In [ ]:
!grpcurl -d '{"experiment":{"name":"exp1", \
               "defaultModel":"iris", \
                "candidates":[\
                    {"modelName":"iris","weight":50},\
                    {"modelName":"iris2","weight":50}\
                    ]}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StartExperiment

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris2"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
testJson("iris", 50)

In [ ]:
!grpcurl -d '{"name":"exp1"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StopExperiment

In [ ]:
testGrpc("iris", 50)

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris2"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!grpcurl -d '{"experiment":{"name":"exp1", \
                "candidates":[{"modelName":"iris","weight":50},{"modelName":"iris2","weight":50}]}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StartExperiment

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris2"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
testJson("iris", 50)

In [ ]:
testExperimentJson("exp1", 50)

In [ ]:
!grpcurl -d '{"name":"exp1"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StopExperiment

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris2"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

## Change Experiment

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris2"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris3"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [ ]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

In [ ]:
!grpcurl -d '{"experiment":{"name":"exp1", \
               "defaultModel":"iris", \
                "candidates":[\
                    {"modelName":"iris","weight":50},\
                    {"modelName":"iris2","weight":50}\
                    ]}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StartExperiment

In [ ]:
testJson("iris", 50)

In [ ]:
testJson("iris2", 50)

In [ ]:
testExperimentJson("exp1", 50)

In [ ]:
!grpcurl -d '{"experiment":{"name":"exp1", \
               "defaultModel":"iris2", \
                "candidates":[\
                    {"modelName":"iris2","weight":50},\
                    {"modelName":"iris3","weight":50}\
                    ]}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StartExperiment

In [ ]:
testJson("iris", 50)

In [ ]:
testJson("iris2", 50)

In [ ]:
!grpcurl -d '{"name":"exp1"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/StopExperiment

In [ ]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris2"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

In [ ]:
!grpcurl -d '{"model":{"name":"iris3"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel